In [1]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [3]:
BASE_DIR = "../../evaluation/output_evals/covost2"
DIRECTION_PAIRS = ['de_en','en_de',
                    'en_zh','es_en',
                    'it_en','pt_en',
                    'zh_en']
SYSTEM_NAMES = ["aya_canary-v2", "aya_owsm4.0-ctc","aya_seamlessm4t","aya_whisper",
                "canary-v2","desta2-8b","gemma_canary-v2","gemma_owsm4.0-ctc","gemma_seamlessm4t",
                "owsm4.0-ctc","phi4multimodal","qwen2audio-7b","seamlessm4t",
                "tower_canary-v2", "tower_owsm4.0-ctc","tower_seamlessm4t","tower_whisper",
                "voxtral-small-24b","whisper"]

In [4]:
def load_results_summaries(base_dir, direction_pairs, system_names):
    """
    Loads all result summaries from a directory structure.

    Args:
        base_dir (str or Path): The base directory for the evaluation outputs.
        direction_pairs (list): A list of language direction strings (e.g., 'en_de').
        system_names (list): A list of system name strings.

    Returns:
        dict: A nested dictionary containing the loaded data, structured as
              {direction: {system: [results]}}.
    """
    base_path = Path(base_dir)
    all_results = {}

    # Use itertools.product to cleanly iterate over all combinations
    for direction, system in itertools.product(direction_pairs, system_names):
        summary_path = base_path / system / direction / 'results_summary.jsonl'
        
        # Initialize the nested dictionary structure
        if direction not in all_results:
            all_results[direction] = {}

        try:
            with summary_path.open('r', encoding='utf-8') as f:
                all_results[direction][system] = [json.loads(line) for line in f]
                
        except FileNotFoundError:
            print(f"Warning: File not found, skipping: {summary_path}")
            all_results[direction][system] = None # Or [] if you prefer an empty list
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {summary_path}: {e}")
            all_results[direction][system] = None

    return all_results

In [5]:
def convert_results_to_dataframe(results_data):
    """
    Converts the nested dictionary of results into a single pandas DataFrame.

    Each row corresponds to a single entry, with 'direction' and 'system'
    columns added, and all 'metrics' unpacked into separate columns.
    """
    all_records = []
    for direction, systems in results_data.items():
        for system, records in systems.items():
            if records is None:
                continue
            for record in records:
                # Separate metrics from the record
                metrics = record.pop("metrics", {})  # safely get metrics
                # Merge everything into one flat dict
                flat_record = {
                    "direction": direction,
                    "system": system,
                    **record,
                    **metrics,  # unpack metrics into top-level keys
                }
                all_records.append(flat_record)

    if not all_records:
        print("No records were found to create a DataFrame.")
        return pd.DataFrame()

    df = pd.DataFrame(all_records)

    # Put identifying info up front
    original_cols = [c for c in df.columns if c not in ["direction", "system"]]
    df = df[["direction", "system"] + original_cols]

    return df

In [6]:
results = load_results_summaries(BASE_DIR, DIRECTION_PAIRS, SYSTEM_NAMES)

In [7]:
df = convert_results_to_dataframe(results)

In [8]:
df

,direction,system,SacreBLEU,chrF,XCOMET,XCOMET-QE,RefMetricX_24,QEMetricX_24,LinguaPy,RefMetricX_24-Strict-linguapy,QEMetricX_24-Strict-linguapy,XCOMET-Strict-linguapy,XCOMET-QE-Strict-linguapy
0,de_en,aya_canary-v2,36.1079,62.0040,0.9087,0.8840,3.5598,3.4758,4.0486,4.4073,4.3276,0.8730,0.8490
1,de_en,aya_owsm4.0-ctc,34.6654,60.8378,0.8866,0.8574,4.0903,3.9701,4.1448,4.9305,4.8147,0.8513,0.8232
2,de_en,aya_seamlessm4t,36.0473,61.9710,0.9072,0.8847,3.6159,3.5270,4.1374,4.4595,4.3784,0.8717,0.8497
3,de_en,aya_whisper,35.4969,61.5708,0.9060,0.8807,3.5921,3.5197,4.1966,4.4707,4.4001,0.8696,0.8453
4,de_en,canary-v2,38.4851,62.3390,0.8699,0.8516,5.0626,4.8247,4.6629,5.9807,5.7577,0.8307,0.8127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,zh_en,tower_canary-v2,0.0509,7.5924,0.1644,0.1793,19.2477,17.9197,71.8865,23.1286,22.9997,0.0438,0.0448
120,zh_en,tower_owsm4.0-ctc,22.2452,51.1634,0.7704,0.8210,5.1971,3.8973,10.5962,7.2328,6.1648,0.6965,0.7340
121,zh_en,tower_seamlessm4t,23.1993,51.9480,0.7878,0.8382,4.6224,3.4815,10.5145,6.6348,5.7066,0.7176,0.7557
122,zh_en,voxtral-small-24b,11.4049,39.9440,0.6633,0.7342,7.4391,5.4210,11.9845,9.3724,7.7433,0.5972,0.6514


In [9]:
for pair in DIRECTION_PAIRS:
    sub_df = df[df['direction']==pair]
    sub_df.to_csv(f"covost2_{pair}.csv",index=False)

In [19]:
sub_df

,direction,system,SacreBLEU,chrF,XCOMET,XCOMET-QE,RefMetricX_24,QEMetricX_24,LinguaPy,RefMetricX_24-Strict-linguapy,QEMetricX_24-Strict-linguapy,XCOMET-Strict-linguapy,XCOMET-QE-Strict-linguapy
12,zh_en,qwen2audio-7b,19.296,47.1774,0.7294,0.7867,5.6995,4.0154,12.5970,7.8198,6.4984,0.6551,0.6934
13,zh_en,phi4multimodal,18.782,39.5620,0.7659,0.8424,5.8845,4.5601,38.4238,12.6944,11.8230,0.4819,0.5144


In [10]:
import glob

In [14]:
import glob
# Read all CSV files for covost2
csv_files = glob.glob("covost2_*.csv")
all_dfs = []

for file in csv_files:
    df_temp = pd.read_csv(file)
    all_dfs.append(df_temp)
# Combine all dataframes
df = pd.concat(all_dfs, ignore_index=True)

# Define the desired column order
desired_order = ['en_de', 'en_zh', 'de_en', 'es_en', 'it_en', 'pt_en', 'zh_en']

# Get all metric columns (exclude direction and system)
metric_columns = [col for col in df.columns if col not in ['direction', 'system']]

print("Available metrics:", metric_columns)

# Create pivot tables for each metric with specified column order
pivot_tables = {}

for metric in metric_columns:
    pivot_df = df.pivot_table(
        index='system',
        columns='direction', 
        values=metric,
        aggfunc='mean'
    )
    
    # Reorder columns to match desired order
    # Only include columns that actually exist in the data
    available_columns = [col for col in desired_order if col in pivot_df.columns]
    pivot_df = pivot_df[available_columns]
    
    # # Add average column
    # pivot_df['average'] = pivot_df.mean(axis=1)
    
    # # Sort by average
    # pivot_df = pivot_df.sort_values('average', ascending=False)
    
    pivot_tables[metric] = pivot_df
    
    # Save each pivot table to CSV
    pivot_df.to_csv(f"covost2_{metric}_pivot.csv")
    
    print(f"\n=== {metric} ===")
    print(pivot_df)

Available metrics: ['en_de', 'en_zh', 'de_en', 'es_en', 'it_en', 'pt_en', 'zh_en', 'average', 'SacreBLEU', 'chrF', 'XCOMET', 'XCOMET-QE', 'RefMetricX_24', 'QEMetricX_24', 'LinguaPy', 'RefMetricX_24-Strict-linguapy', 'QEMetricX_24-Strict-linguapy', 'XCOMET-Strict-linguapy', 'XCOMET-QE-Strict-linguapy']

=== en_de ===
Empty DataFrame
Columns: []
Index: []

=== en_zh ===
Empty DataFrame
Columns: []
Index: []

=== de_en ===
Empty DataFrame
Columns: []
Index: []

=== es_en ===
Empty DataFrame
Columns: []
Index: []

=== it_en ===
Empty DataFrame
Columns: []
Index: []

=== pt_en ===
Empty DataFrame
Columns: []
Index: []

=== zh_en ===
Empty DataFrame
Columns: []
Index: []

=== average ===
Empty DataFrame
Columns: []
Index: []

=== SacreBLEU ===
direction            en_de    en_zh    de_en    es_en    it_en    pt_en  \
system                                                                    
aya_canary-v2      32.7329  36.4145  36.1079  39.1207  33.4437  47.2553   
aya_owsm4.0-ctc    31.5364 

/var/folders/2x/p1hwp9tj16v7r_c_lnx6xc6rbpr63k/T/ipykernel_23094/2443696842.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(all_dfs, ignore_index=True)
